In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers.merge import concatenate
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Activation, merge, Input, Lambda, Reshape
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional
from keras.utils.np_utils import to_categorical
from keras import initializers
from keras import backend as K
from keras.engine.topology import Layer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import sklearn
import keras
from sklearn.preprocessing import OneHotEncoder
#X_train, X_test, y_train, y_test = train_test_split(title, label, test_size=0.1, random_state=42)

'''import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))'''


C:\Users\28529\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'import tensorflow as tf\nfrom keras.backend.tensorflow_backend import set_session\nconfig = tf.ConfigProto()\nconfig.gpu_options.per_process_gpu_memory_fraction = 0.5\nset_session(tf.Session(config=config))'

In [2]:
def getOnePredict(source):
    result = []
    for i in source:
        result_item = np.zeros(i.size)
        result_item[i.argmax()] = 1
        result.append(result_item)
    return np.array(result)

In [3]:
def onehot2index(source):
    result = np.zeros(source.shape[0])
    for i in range(source.shape[0]):
        result[i] = source[i].argmax()
    return result

In [4]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

class NameDic:#注意最后一层的list不是NameDic实例，而是数字
    
    def __init__(self, name):
        self.son = []
        self.name = name
        
    def findSon(self, name):
        for i in self.son:
            if i.name == name:
                return i
        return None
    
    def haveSuchSon(self, name):
        for i in self.son:
            if i.name == name:
                return True
        return False

def buildFatherTree():
    
    root = NameDic('0')
    for i in range(10):
        root.son.append(NameDic(str(i+1)))
    
    with open('train_a_c.txt') as f:
        isFirst = True
        count = 1
        while True:
            line = f.readline()

            if isFirst:
                isFirst = False
                continue

            if not line:
                break

            l = line.split('\t')

            labelA = l[5]
            labelB = l[6]
            labelC = l[7]
            
            nodeA = root.findSon(labelA)
            if not nodeA.haveSuchSon(labelB):
                temp = NameDic(labelB)
                nodeA.son.append(temp)
                temp.son.append(labelC)
                
            else:
                temp = nodeA.findSon(labelB)
                temp.son.append(labelC)
            
    return root

def parseSecond(root,labelSecond):
    label = []
    for i in labelSecond:
        label.append(i.argmax()+1)
    
    return np.array(label)

root = buildFatherTree()

In [5]:
#get word level data
def getData():
    label = []
    data = []
    firstLabel = []
    
    with open('train_a_c.txt') as f:
        isFirst = True
        count = 1
        while True:
            line = f.readline()

            if isFirst:
                isFirst = False
                continue            

            if not line:
                break

            l = line.split('\t')
            
            firstLabel.append(int(l[5]))
            label.append(int(l[6]))
            title = l[2].split(',')#e.g. ['w12','w23']
            content = l[4].split(',')
            new_result = np.zeros(187)
            
            #format to delete 'w'
            for i in range(len(title)):
                new_result[i] = title[i][1:]

            for i in range(len(title),len(content)+len(title)):
                new_result[i] = content[i-len(title)][1:]
                        
            data.append(new_result)
            if count%10000 == 0:
                print(count)
            count+=1
    
    #change lable to one hot
    label = np.array(label)
    ohe = OneHotEncoder()
    example = []
    for i in range(11,11+64):
        example.append([i])
    ohe.fit(example)
    label = ohe.transform(label.reshape(label.size,1)).toarray()
    
    #change lable to one hot
    firstLabel = np.array(firstLabel)
    ohe = OneHotEncoder()
    example = []
    for i in range(1,11):
        example.append([i])
    ohe.fit(example)
    firstLabel = ohe.transform(firstLabel.reshape(firstLabel.size,1)).toarray()
    
    return (np.array(data),label,firstLabel)


In [6]:
(data,label,firstLabel) = getData()

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


In [10]:
#随机切分
def split(source, testRate):
    want = int(len(source[0])*testRate)
    result = []
    for i in source:
        a_temp = []
        b_temp = np.array(np.copy(i))
        
        for _ in range(want): 
            index = np.random.randint(0,b_temp.shape[0])
            a_temp.append(i[index])
            b_temp = np.delete(b_temp, index, axis=0)
        
        result.append(b_temp)
        result.append(np.array(a_temp))
        
        
    return result#all new numpy array


[x_train, x_test, y_train, y_test,z_train,z_test] = split([data,label,firstLabel],0.01)

#from sklearn.cross_validation import train_test_split
#import keras
#x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.1, random_state=42)


KeyboardInterrupt: 

In [11]:
firstLabel.shape

(140561, 10)

In [ ]:
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [12]:
from keras.layers import BatchNormalization

main_input = Input(shape=(187,), dtype='float64')
embedder = Embedding(300 + 1, 256, input_length = 187)
embed = embedder(main_input)

# cnn1模块，kernel_size = 3
conv1_1 = Convolution1D(256, 3, padding='same')(embed)
bn1_1 = BatchNormalization()(conv1_1)
relu1_1 = Activation('relu')(bn1_1)
conv1_2 = Convolution1D(128, 3, padding='same')(relu1_1)
bn1_2 = BatchNormalization()(conv1_2)
relu1_2 = Activation('relu')(bn1_2)
cnn1 = MaxPool1D(pool_size=4)(relu1_2)

# cnn2模块，kernel_size = 4
conv2_1 = Convolution1D(256, 4, padding='same')(embed)
bn2_1 = BatchNormalization()(conv2_1)
relu2_1 = Activation('relu')(bn2_1)
conv2_2 = Convolution1D(128, 4, padding='same')(relu2_1)
bn2_2 = BatchNormalization()(conv2_2)
relu2_2 = Activation('relu')(bn2_2)
cnn2 = MaxPool1D(pool_size=4)(relu2_2)

# cnn3模块，kernel_size = 5
conv3_1 = Convolution1D(256, 5, padding='same')(embed)
bn3_1 = BatchNormalization()(conv3_1)
relu3_1 = Activation('relu')(bn3_1)
conv3_2 = Convolution1D(128, 5, padding='same')(relu3_1)
bn3_2 = BatchNormalization()(conv3_2)
relu3_2 = Activation('relu')(bn3_2)
cnn3 = MaxPool1D(pool_size=4)(relu3_2)

# 拼接三个模块
cnn = concatenate([cnn1,cnn2,cnn3], axis=-1)
flat = Flatten()(cnn)

# 此处concantenate上强监督
aux_input = Input(shape=(10,), dtype='float32')

flat_2 = concatenate([aux_input,flat])

drop = Dropout(0.5)(flat_2)
fc = Dense(512)(drop)
bn = BatchNormalization()(fc)
main_output = Dense(64, activation='softmax')(bn)

model = Model(inputs = [main_input,aux_input], outputs = main_output)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[f1])
'''
model.fit(x={'main_input': main_data, 'aux_input': aux_data},
          y={'main_output': main_labels, 'aux_output': main_labels},
          batch_size=32, epochs=10,verbose=1)
'''



history = model.fit([data, firstLabel], 
                    [label],
                    batch_size=64,
                    epochs=4)
                    #validation_data=({'main_input': x_test, 'aux_input': z_test.reshape(z_test.size,1)}, y_test))
                    #validation_data=(x_test,y_test))

Epoch 1/4
140561/140561 [==============================] - 278s 2ms/step - loss: 2.0565 - f1: 0.4362
Epoch 2/4
140561/140561 [==============================] - 274s 2ms/step - loss: 1.0868 - f1: 0.6713
Epoch 3/4
140561/140561 [==============================] - 274s 2ms/step - loss: 0.9763 - f1: 0.7026
Epoch 4/4
140561/140561 [==============================] - 274s 2ms/step - loss: 0.9078 - f1: 0.7201


In [13]:
model.save('E:history2.h5')

In [14]:
while True:
    history = model.fit([data, firstLabel], 
                    [label],
                    batch_size=64,
                    epochs=2)
    name = 'E:history2_' + str(history.history['f1'][-1])[2:]+'.h5'
    model.save(name)
    

Epoch 1/2
140561/140561 [==============================] - 275s 2ms/step - loss: 0.8610 - f1: 0.7340
Epoch 2/2
140561/140561 [==============================] - 275s 2ms/step - loss: 0.8265 - f1: 0.7441
Epoch 1/2
140561/140561 [==============================] - 274s 2ms/step - loss: 0.8618 - f1: 0.7332
Epoch 2/2
140561/140561 [==============================] - 274s 2ms/step - loss: 0.8092 - f1: 0.7479
Epoch 1/2
140561/140561 [==============================] - 275s 2ms/step - loss: 0.7844 - f1: 0.7561
Epoch 2/2
140561/140561 [==============================] - 274s 2ms/step - loss: 0.7362 - f1: 0.7701
Epoch 1/2
140561/140561 [==============================] - 273s 2ms/step - loss: 0.6969 - f1: 0.7807
Epoch 2/2
140561/140561 [==============================] - 265s 2ms/step - loss: 0.6708 - f1: 0.7896
Epoch 1/2
140561/140561 [==============================] - 268s 2ms/step - loss: 0.6606 - f1: 0.7917
Epoch 2/2
140561/140561 [==============================] - 274s 2ms/step - loss: 0.6623 - f

KeyboardInterrupt: 

In [22]:
model = keras.models.load_model('E:history2_770104540397592.h5', custom_objects={"f1": f1})
#test on valid data
#get word level data
def getValData():
    label = []
    data = []
    firstLabel = []
    
    with open('valid_a_c.txt') as f:
        isFirst = True
        count = 1
        while True:
            line = f.readline()

            if isFirst:
                isFirst = False
                continue            

            if not line:
                break

            l = line.split('\t')
            
            firstLabel.append(int(l[5]))
            label.append(int(l[6]))
            title = l[2].split(',')#e.g. ['w12','w23']
            content = l[4].split(',')
            new_result = np.zeros(187)
            
            #format to delete 'w'
            for i in range(len(title)):
                new_result[i] = title[i][1:]

            for i in range(len(title),len(content)+len(title)):
                new_result[i] = content[i-len(title)][1:]
                        
            data.append(new_result)
            if count%10000 == 0:
                print(count)
            count+=1
    
    #change lable to one hot
    label = np.array(label)
    ohe = OneHotEncoder()
    example = []
    for i in range(11,11+64):
        example.append([i])
    ohe.fit(example)
    label = ohe.transform(label.reshape(label.size,1)).toarray()
    
    #change lable to one hot
    firstLabel = np.array(firstLabel)
    ohe = OneHotEncoder()
    example = []
    for i in range(1,11):
        example.append([i])
    ohe.fit(example)
    firstLabel = ohe.transform(firstLabel.reshape(firstLabel.size,1)).toarray()
    
    return (np.array(data),label,firstLabel)


(valid_data, valid_label, valid_first_label) = getValData()
predict_label = model.predict([valid_data,valid_first_label])
print(onehot2index(predict_label))
print(onehot2index(valid_label))
sklearn.metrics.f1_score(onehot2index(valid_label),onehot2index(predict_label),average='macro')

10000
[63. 38. 33. ... 29. 29. 29.]
[28. 28. 28. ... 29. 29. 29.]


C:\Users\28529\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.6672961331528969

In [24]:
def getOutPutAndSave(model):
    data = []
    with open('test_a.txt') as f:
        isFirst = True
        while True:
            line = f.readline()

            if isFirst:
                isFirst = False
                continue

            if not line:
                break

            l = line.split('\t')

            title = l[2].split(',')#e.g. ['w12','w23']
            content = l[4].split(',')
            new_result = np.zeros(187)
            
            #format to delete 'w'
            for i in range(len(title)):
                new_result[i] = title[i][1:]

            for i in range(len(title),len(content)+len(title)):
                new_result[i] = content[i-len(title)][1:]
            
            data.append(new_result)
        
    data = np.array(data)
    
    firstLabel = []
    with open('Second.txt') as f:
        while True:
            line = f.readline()
            if not line:
                break
                
            firstLabel.append(np.array(int(line[:-1])))
    
    #change lable to one hot
    firstLabel = np.array(firstLabel)
    ohe = OneHotEncoder()
    example = []
    for i in range(11,11+64):
        example.append([i])
    ohe.fit(example)
    firstLabel = ohe.transform(firstLabel.reshape(firstLabel.size,1)).toarray()
    
    print(data.shape)
    print(firstLabel.shape)
    
    result = model.predict([data, firstLabel])
    
    target = open('Second.txt','w')
    
    for i in result:
        target.write(str(i.argmax()+1)+'\n')#输出
        
    target.close()

getOutPutAndSave(model)

(12500, 187)
(1, 10)


IndexError: index 1 is out of bounds for axis 0 with size 1